In [2]:
import time
import datetime
import requests
import json
import pandas as pd

from pandas import json_normalize
from sqlalchemy import create_engine

In [3]:
API_KEY = '5a0xmKOGOTI1G248OJ9GOL3GD9ohbQ0W'

In [4]:
def get_uri(query: str, pages_num: str, date: str, api_key: str) -> str:
    uri = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}'
    uri = uri + f'&page={pages_num}&begin_date={date}&end_date={date}'
    uri = uri + f'&api-key={api_key}'
    return uri

In [5]:
df = pd.DataFrame()
current_date = datetime.datetime.now() - datetime.timedelta(days=3)
page_num = 1

In [6]:
while True:
    URI = get_uri(query='election', pages_num=str(page_num), date=current_date.strftime('%Y%m%d'), api_key=API_KEY)
    response = requests.get(URI)
    data = response.json()

    # print(f'GET page {page_num}: ')
    # print(data)

    try:
        df_request = json_normalize(data['response'], record_path=['docs'])
        if df_request.empty:
            break
    except:
        break

    df = pd.concat([df_request, df])
    time.sleep(20)
    page_num += 1

In [7]:
# Finding duplicates. And delete them.
if len(df['_id'].unique()) < len(df):
    print('There are duplicates in the data')
    df = df.drop_duplicates('_id', keep='first')

# Finding and delete articles without titles.
if df['headline.main'].isnull().any():
    print('There are missing values in this dataset')
    df = df[df['headline.main'].isnull() == False]

# Filtered op-ed articles.
df = df[df['type_of_material'] != 'op-ed']

# Stay just four columns
df = df[['headline.main', 'pub_date', 'byline.original', 'web_url']]

# Rename columns
df.columns = ['headline', 'date', 'author', 'url']

print(df.head(12))

                                            headline  \
0  Transcript: Ezra Klein Interviews Michael Podh...   
1  House Ethics Panel Will Not Push to Expel Geor...   
2                                 Wednesday Briefing   
3  Top German Journalist Received €600,000 From P...   
4  U.K. Can’t Deport Asylum Seekers to Rwanda, Su...   
5        U.S. Presses China to Stop Flow of Fentanyl   
6  Biden-Xi Talks Lead to Little but a Promise to...   
7                              The Cost of Streaming   
8  Why a Boston Tea Party Patriot Is Being Honore...   
9   The Supreme Court’s New Ethics Code: ‘Worthless’   

                       date                                author  \
0  2023-11-15T05:32:49+0000                                  None   
1  2023-11-15T19:48:04+0000  By Grace Ashford and Luke Broadwater   
2  2023-11-15T04:24:01+0000                      By Natasha Frost   
3  2023-11-15T20:29:59+0000                      By Graham Bowley   
4  2023-11-15T05:01:20+0000           